In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [4]:
file = '/home/ankita/resource/books/NYCrime.csv'

In [5]:
data = sc.textFile(file)

In [18]:
#Extract only header
header = data.first()
print(header)

OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1


In [20]:
#Extract only first row without header
rows = data.filter(lambda x : x != header) #returns pipelined RDD
rows.first()

'1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"'

In [22]:
rows.map(lambda x: x.split(",")).first()

['1',
 'f070032d',
 '09/06/1940 07:30:00 PM',
 'Friday',
 'Sep',
 '6',
 '1940',
 '19',
 '9',
 '7',
 '2010',
 'BURGLARY',
 'FELONY',
 'D',
 '66',
 'BROOKLYN',
 'N.Y. POLICE DEPT',
 '987478',
 '166141',
 '"(40.6227027620001',
 ' -73.9883732929999)"']

In [23]:
import csv
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
from collections import namedtuple

In [24]:
fields = header.replace(" ","_").replace("/",'_').split(",")
print(fields)

['OBJECTID', 'Identifier', 'Occurrence_Date', 'Day_of_Week', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Year', 'Occurrence_Hour', 'CompStat_Month', 'CompStat_Day', 'CompStat_Year', 'Offense', 'Offense_Classification', 'Sector', 'Precinct', 'Borough', 'Jurisdiction', 'XCoordinate', 'YCoordinate', 'Location_1']


In [25]:
Crime = namedtuple("Crime", fields, verbose=True)

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Crime(tuple):
    'Crime(OBJECTID, Identifier, Occurrence_Date, Day_of_Week, Occurrence_Month, Occurrence_Day, Occurrence_Year, Occurrence_Hour, CompStat_Month, CompStat_Day, CompStat_Year, Offense, Offense_Classification, Sector, Precinct, Borough, Jurisdiction, XCoordinate, YCoordinate, Location_1)'

    __slots__ = ()

    _fields = ('OBJECTID', 'Identifier', 'Occurrence_Date', 'Day_of_Week', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Year', 'Occurrence_Hour', 'CompStat_Month', 'CompStat_Day', 'CompStat_Year', 'Offense', 'Offense_Classification', 'Sector', 'Precinct', 'Borough', 'Jurisdiction', 'XCoordinate', 'YCoordinate', 'Location_1')

    def __new__(_cls, OBJECTID, Identifier, Occurrence_Date, Day_of_Week, Occurrence_Month, Occurrence_Day, Occurrence_Year, Occurrence_Hour, CompStat_Month, CompStat_Day, CompStat_Year, Offens

In [26]:
def make(row):
    reader = csv.reader(StringIO(row))
    row = next(reader)
    return Crime(*row)

In [27]:
crimes = rows.map(make)

In [28]:
crimes.first()

Crime(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')

In [29]:
crimes.first().Offense

'BURGLARY'

In [30]:
crimes.map(lambda x: x.Offense).countByValue()

defaultdict(int,
            {'BURGLARY': 191369,
             'GRAND LARCENY': 428993,
             'GRAND LARCENY OF MOTOR VEHICLE': 101963,
             'RAPE': 13779,
             'ROBBERY': 198744,
             'FELONY ASSAULT': 184042,
             'MURDER & NON-NEGL. MANSLAUGHTE': 4574,
             'NA': 1})

In [31]:
crimeFiltered = crimes.filter(lambda x: not(x.Offense == 'NA' or x.Occurrence_Year == '')) \
                       .filter(lambda x: int(x.Occurrence_Year) >= 2006)

In [32]:
crimeFiltered.map(lambda x: x.Occurrence_Year).countByValue()

defaultdict(int,
            {'2006': 127887,
             '2015': 102657,
             '2007': 120554,
             '2008': 117375,
             '2009': 106018,
             '2010': 105643,
             '2011': 107206,
             '2012': 111798,
             '2013': 111286,
             '2014': 106849})

In [33]:
def extractCoords(locations):
    loc_latt = float(locations[1:locations.index(",")])
    loc_long = float(locations[locations.index(",")+1:-1])
    return (loc_latt, loc_long)


In [34]:
#Finding maximum and minimum location lattitude and longitude values in our code
crimeFiltered.map(lambda x : extractCoords(x.Location_1)) \
            .reduce(lambda x,y : (min(x[0], y[0]), min(x[1], y[1])))

(40.112709974, -77.519206334)

In [35]:
crimeFiltered.first()

Crime(OBJECTID='258', Identifier='13b6949b', Occurrence_Date='01/09/2006 12:00:00 AM', Day_of_Week='Monday', Occurrence_Month='Jan', Occurrence_Day='9', Occurrence_Year='2006', Occurrence_Hour='0', CompStat_Month='8', CompStat_Day='16', CompStat_Year='2006', Offense='GRAND LARCENY', Offense_Classification='FELONY', Sector='H', Precinct='102', Borough='QUEENS', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='1029007', YCoordinate='194256', Location_1='(40.6997596520001, -73.8385879319999)')

In [37]:
crimeFiltered.map(lambda x : extractCoords(x.Location_1)).first()

(40.6997596520001, -73.8385879319999)

In [39]:

crimeFiltered.map(lambda x : extractCoords(x.Location_1)).first()


(40.6997596520001, -73.8385879319999)

In [41]:
crimeFiltered.map(lambda x: (x.Location_1 == '')).countByValue()

defaultdict(int, {False: 1117273})